In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git 
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -U adapter-transformers
!pip install -U --user neptune transformers datasets evaluate torch scipy scikit-learn numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TrainerCallback, AutoModelForCausalLM, AutoModelForSeq2SeqLM, AdapterTrainer, EvalPrediction
from datasets import load_dataset, concatenate_datasets, load_from_disk, load_metric, Dataset, ClassLabel
import numpy as np
import pandas as pd
import os
import neptune

In [ ]:
import os
from getpass import getpass
api_token = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI2ZmViNmNiNy1hYmFiLTRhOWYtYjA3NC02NDRhYjJmZjUxODcifQ=='
project = 'dducl/Dissertation'
os.environ["NEPTUNE_API_TOKEN"] = getpass("Enter your Neptune API token: ")
os.environ["NEPTUNE_PROJECT"] = "dducl/Dissertation"

Enter your Neptune API token: ··········


In [ ]:
# Load data
df_total = pd.read_parquet('/content/drive/MyDrive/Dissertation/df_total.parquet')

In [ ]:
# One-shot sample
df_label_unique_sample = df_total.groupby('label_cat', group_keys=False).apply(lambda df: df.sample(1))

# Create label mapping
label = list(df_label_unique_sample['label'])
id = list(df_label_unique_sample['label_cat'])
label_to_id = dict(zip(label,id))
id_to_label = dict(zip(id,label))

In [ ]:
# Define mapping
Mapping = ClassLabel(names= label)

# Load dataset
# Load the parquet dataset
dataset = load_dataset("parquet", data_files={'train': '/content/drive/MyDrive/Dissertation/train.parquet', 'test': '/content/drive/MyDrive/Dissertation/test.parquet','valid':'/content/drive/MyDrive/Dissertation/validation.parquet'})

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Rename columns to the suitable column names for modelling
test_ds = dataset['test']
train_ds = dataset['train']
valid_ds = dataset['valid']
train_ds = train_ds.rename_column("Text", "text")
train_ds = train_ds.rename_column("label_cat", "labels")
test_ds = test_ds.rename_column("Text", "text")
test_ds= test_ds.rename_column("label_cat", "labels")
valid_ds = valid_ds.rename_column("Text", "text")
valid_ds= valid_ds.rename_column("label_cat", "labels")

In [ ]:
model_name = 'roberta-base'
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)
def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=150, truncation=True, padding="max_length")

In [ ]:
# Encode the input training data
train_ds = train_ds.map(encode_batch, batched=True)
# Transform to pytorch tensors and only output the required columns
train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Encode the input test data
test_ds = test_ds.map(encode_batch, batched=True)
# Transform to pytorch tensors and only output the required columns
test_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

In [ ]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    model_name,
    num_labels=len(label),
)
model = RobertaModelWithHeads.from_pretrained(
    model_name,
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/roberta/adapter_model.py:274: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/roberta/adapter_model.py:252: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForP

In [ ]:
# Add a new adapter
model.add_adapter("baseline")
model.add_classification_head(
    "baseline",
    num_labels= len(label),
    id2label=id_to_label
  )
# Activate the adapter
model.train_adapter("baseline")

In [ ]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    f"{model_name}-baseline_adapter",
    evaluation_strategy = 'epoch',
    save_strategy="epoch",
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="neptune",
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_accuracy,
)

/usr/local/lib/python3.10/dist-packages/transformers/integrations.py:1114: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  from neptune.new.metadata_containers.run import Run


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 567
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 216
  Number of trainable parameters = 1509728
/usr/local/lib/python3.10/dist-packages/transformers/integrations.py:1164: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  self._run = init_run(**self._init_run_kwargs, **additional_neptune

https://app.neptune.ai/dducl/Dissertation/e/DIS-2


/usr/local/lib/python3.10/dist-packages/transformers/integrations.py:1221: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'NoneType'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  self._metadata_namespace[NeptuneCallback.model_parameters_key] = model.config.to_dict()
/usr/local/lib/python3.10/dist-packages/transformers/integrations.py:1221: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'list'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  self._

Epoch,Training Loss,Validation Loss,Acc
1,No log,3.189548,0.209877
2,No log,3.033872,0.209877
3,No log,2.824613,0.234568
4,No log,2.561463,0.296296
5,No log,2.433053,0.382716
6,No log,2.382947,0.395062


***** Running Evaluation *****
  Num examples = 81
  Batch size = 16
Saving model checkpoint to roberta-base-baseline_adapter/checkpoint-36
Configuration saved in roberta-base-baseline_adapter/checkpoint-36/baseline/adapter_config.json
Module weights saved in roberta-base-baseline_adapter/checkpoint-36/baseline/pytorch_adapter.bin
Configuration saved in roberta-base-baseline_adapter/checkpoint-36/baseline/head_config.json
Module weights saved in roberta-base-baseline_adapter/checkpoint-36/baseline/pytorch_model_head.bin
Configuration saved in roberta-base-baseline_adapter/checkpoint-36/baseline/head_config.json
Module weights saved in roberta-base-baseline_adapter/checkpoint-36/baseline/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 81
  Batch size = 16
Saving model checkpoint to roberta-base-baseline_adapter/checkpoint-72
Configuration saved in roberta-base-baseline_adapter/checkpoint-72/baseline/adapter_config.json
Module weights saved in roberta-base-baseline

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 12 operations to synchronize with Neptune. Do not kill this process.
All 12 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/dducl/Dissertation/e/DIS-2/metadata


TrainOutput(global_step=216, training_loss=2.7729257654260704, metrics={'train_runtime': 4194.0347, 'train_samples_per_second': 0.811, 'train_steps_per_second': 0.052, 'total_flos': 266855220576000.0, 'train_loss': 2.7729257654260704, 'epoch': 6.0})

## Adpater fusion
As the previous baseline adapter performs not too well, let us examine whether use existing adpater and fusion will help to enhance the performance of the model

In [ ]:
# Use the same model as before
model2 = RobertaModelWithHeads.from_pretrained(
    model_name,
    config=config,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferen

In [ ]:
from transformers.adapters.composition import Fuse
# Load the pre-trained adapters we want to fuse
model2.add_adapter('baseline_2')
model2.load_adapter("AdapterHub/roberta-base-pf-pmb_sem_tagging",load_as="pmbst", source="hf", with_head=False)
model2.load_adapter("nli/multinli@ukp", load_as="multinli", with_head=False)
# Add a fusion layer for all loaded adapters
model2.add_adapter_fusion(Fuse("pmbst", "multinli",'baseline_2'))
model2.set_active_adapters(Fuse("pmbst", "multinli",'baseline_2'))
model2.add_classification_head(
    "baseline_2",
    num_labels= len(label),
    id2label=id_to_label,
    overwrite_ok=True
  )
# Unfreeze and activate fusion setup
adapter_setup = Fuse("pmbst", "multinli",'baseline_2')
model2.train_adapter_fusion(adapter_setup)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Overwriting existing adapter 'multinli'.


In [ ]:
training_args = TrainingArguments(
    f"model2-adapter_fushion",
    evaluation_strategy = 'epoch',
    save_strategy="epoch",
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="neptune",
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer2 = AdapterTrainer(
    model=model2,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_accuracy,
)

/usr/local/lib/python3.10/dist-packages/transformers/integrations.py:1114: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  from neptune.new.metadata_containers.run import Run


In [ ]:
trainer2.train()

***** Running training *****
  Num examples = 567
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 216
  Number of trainable parameters = 21867296


https://app.neptune.ai/dducl/Dissertation/e/DIS-4


/usr/local/lib/python3.10/dist-packages/transformers/integrations.py:1221: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'NoneType'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  self._metadata_namespace[NeptuneCallback.model_parameters_key] = model.config.to_dict()
/usr/local/lib/python3.10/dist-packages/transformers/integrations.py:1221: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'list'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  self._

Epoch,Training Loss,Validation Loss,Acc
1,No log,3.023161,0.296296
2,No log,2.658662,0.296296
3,No log,2.397679,0.333333
4,No log,2.224698,0.407407
5,No log,2.130226,0.456790
6,No log,2.087685,0.444444


***** Running Evaluation *****
  Num examples = 81
  Batch size = 16
Saving model checkpoint to model2-adapter_fushion/checkpoint-36
Configuration saved in model2-adapter_fushion/checkpoint-36/pmb/adapter_config.json
Module weights saved in model2-adapter_fushion/checkpoint-36/pmb/pytorch_adapter.bin
Configuration saved in model2-adapter_fushion/checkpoint-36/multinli/adapter_config.json
Module weights saved in model2-adapter_fushion/checkpoint-36/multinli/pytorch_adapter.bin
Configuration saved in model2-adapter_fushion/checkpoint-36/baseline_2/adapter_config.json
Module weights saved in model2-adapter_fushion/checkpoint-36/baseline_2/pytorch_adapter.bin
Configuration saved in model2-adapter_fushion/checkpoint-36/baseline_2/head_config.json
Module weights saved in model2-adapter_fushion/checkpoint-36/baseline_2/pytorch_model_head.bin
Configuration saved in model2-adapter_fushion/checkpoint-36/pmbst/adapter_config.json
Module weights saved in model2-adapter_fushion/checkpoint-36/pmbst/

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 6 operations to synchronize with Neptune. Do not kill this process.
All 6 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/dducl/Dissertation/e/DIS-4/metadata


TrainOutput(global_step=216, training_loss=2.481488263165509, metrics={'train_runtime': 7800.6237, 'train_samples_per_second': 0.436, 'train_steps_per_second': 0.028, 'total_flos': 340141485600000.0, 'train_loss': 2.481488263165509, 'epoch': 6.0})

In [ ]:
#save trainer
model2.save_adapter_fusion("/content/drive/MyDrive/Dissertation/adapter_transformer/roberta-base-baseline_adapter", "pmbst,multinli,baseline_2")
model2.save_all_adapters("/content/drive/MyDrive/Dissertation/adapter_transformer/roberta-base-baseline_adapter")

Configuration saved in /content/drive/MyDrive/Dissertation/adapter_transformer/roberta-base-baseline_adapter/adapter_fusion_config.json
Module weights saved in /content/drive/MyDrive/Dissertation/adapter_transformer/roberta-base-baseline_adapter/pytorch_model_adapter_fusion.bin
Configuration saved in /content/drive/MyDrive/Dissertation/adapter_transformer/roberta-base-baseline_adapter/pmb/adapter_config.json
Module weights saved in /content/drive/MyDrive/Dissertation/adapter_transformer/roberta-base-baseline_adapter/pmb/pytorch_adapter.bin
Configuration saved in /content/drive/MyDrive/Dissertation/adapter_transformer/roberta-base-baseline_adapter/multinli/adapter_config.json
Module weights saved in /content/drive/MyDrive/Dissertation/adapter_transformer/roberta-base-baseline_adapter/multinli/pytorch_adapter.bin
Configuration saved in /content/drive/MyDrive/Dissertation/adapter_transformer/roberta-base-baseline_adapter/baseline_2/adapter_config.json
Module weights saved in /content/driv